# [ 가위바위보 분류기 ]

## Step_0. 필요한 패키지 import

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import os, glob

print("PIL library import complete")

PIL library import complete


## Step_1. train data 준비

In [2]:
# x_train 이미지 28x28 사이즈로 변경

import os
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME")+"/practice/rock_scissor_paper"
print("=> 이미지 디렉토리 경로:", image_dir_path)

images_scissor = glob.glob(image_dir_path + "/train_set/scissor_jpg/*.*")
images_rock = glob.glob(image_dir_path + "/train_set/rock_jpg/*.*")
images_paper = glob.glob(image_dir_path + "/train_set/paper_jpg/*.*")

# 파일마다 모두 28X28 사이즈로 바꾸어 저장
target_size = (28, 28)
for img in images_scissor:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("가위 이미지 resize 완료!")

for img in images_rock:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("바위 이미지 resize 완료!")

for img in images_paper:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("보 이미지 resize 완료!")

=> 이미지 디렉토리 경로: /home/myungjin-kim/practice/rock_scissor_paper
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


In [6]:
# y_train 라벨링

def load_data(img_path):
    
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data = 3080   # 총 이미지 개수
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper_jpg/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/practice/rock_scissor_paper/train_set"
(x_train, y_train) = load_data(image_dir_path)

x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3080 입니다.
x_train shape: (3080, 28, 28, 3)
y_train shape: (3080,)


In [9]:
print('x_train의 최솟값:', np.min(x_train), ', x_train의 최댓값:', np.max(x_train))

print('x_train_norm의 최솟값:', np.min(x_train_norm), ', x_train_norm의 최댓값:', np.max(x_train_norm))

x_train의 최솟값: 0 , x_train의 최댓값: 255
x_train_norm의 최솟값: 0.0 , x_train_norm의 최댓값: 1.0


## Step_2. model 설계

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?

n_channel_1 = 32
n_channel_2 = 64
n_dense = 32
n_train_epoch = 15

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                51232     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

## Step_3. model training

In [12]:
# model.compile(), model.fit()

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs = n_train_epoch)


Epoch 1/15
97/97 [==============================] - 1s 10ms/step - loss: 0.6400 - accuracy: 0.6127
Epoch 2/15
97/97 [==============================] - 1s 10ms/step - loss: 0.3794 - accuracy: 0.8201
Epoch 3/15
97/97 [==============================] - 1s 10ms/step - loss: 0.2754 - accuracy: 0.8766
Epoch 4/15
97/97 [==============================] - 1s 11ms/step - loss: 0.2140 - accuracy: 0.9058
Epoch 5/15
97/97 [==============================] - 1s 11ms/step - loss: 0.1739 - accuracy: 0.9224
Epoch 6/15
97/97 [==============================] - 1s 10ms/step - loss: 0.1428 - accuracy: 0.9393
Epoch 7/15
97/97 [==============================] - 1s 11ms/step - loss: 0.1199 - accuracy: 0.9516
Epoch 8/15
97/97 [==============================] - 1s 10ms/step - loss: 0.0952 - accuracy: 0.9597
Epoch 9/15
97/97 [==============================] - 1s 10ms/step - loss: 0.0814 - accuracy: 0.9666
Epoch 10/15
97/97 [==============================] - 1s 10ms/step - loss: 0.0669 - accuracy: 0.9740
Epoch 11/

## Step_4. test data 준비

In [13]:
# 이미지를 불러와서 28x28 사이즈로 변경할 겁니다. 

import os
# 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME")+"/practice/rock_scissor_paper"
print("=> 이미지 디렉토리 경로:", image_dir_path)

images_scissor = glob.glob(image_dir_path + "/test_set/scissor/*.jpg")
images_rock = glob.glob(image_dir_path + "/test_set/rock/*.jpg")
images_paper = glob.glob(image_dir_path + "/test_set/paper/*.jpg")

# 파일마다 모두 28X28 사이즈로 바꾸어 저장
target_size = (28, 28)

for img in images_scissor:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("가위 이미지 resize 완료!")

for img in images_rock:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("바위 이미지 resize 완료!")

for img in images_paper:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img,"JPEG")
print("보 이미지 resize 완료!")


=> 이미지 디렉토리 경로: /home/myungjin-kim/practice/rock_scissor_paper
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


In [17]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data = 311  # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size = 28
    color = 3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/practice/rock_scissor_paper/test_set"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/208.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 311 입니다.
x_test shape: (311, 28, 28, 3)
y_test shape: (311,)


In [18]:
print('x_test의 최솟값:', np.min(x_test), ', x_test의 최댓값:', np.max(x_test))

print('x_test_norm의 최솟값:', np.min(x_test_norm), ', x_test_norm의 최댓값:', np.max(x_test_norm))

x_test의 최솟값: 0 , x_test의 최댓값: 208
x_test_norm의 최솟값: 0.0 , x_test_norm의 최댓값: 1.0


## Step_5. test

In [19]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print()

10/10 - 0s - loss: 5.9083 - accuracy: 0.6592



## [ 진행사항 ]

1. 1차시도

train data = 3080 (정규화 완료) / test data = 311 (정규화 완료)

n_channel_1 = 32
n_channel_2 = 64
n_dense = 32
n_train_epoch = 15

=> result: loss: 5.9083 / accuracy: 0.6592


2. 2차시도

train data = 3080 (정규화 완료) / test data = 311 (정규화 완료)

n_channel_1 = 32
n_channel_2 = 64
n_dense = 32
n_train_epoch = 15

=> result: loss: 5.9083 / accuracy: 0.6592

## [ 회 고 ]

1. 이번 프로젝트에서 어려웠던 점


2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.

- 알아낸 점:

- 모호한 점: vali

3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.


4. 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정.
자기 다짐